In [11]:
import torch

from cellular_automata import GameOfLife
from common import Data

In [12]:
N_FEATURES = 16
SHAPE = (4, 4)
N_RANGE = 2**N_FEATURES

In [ ]:
# for now label is just one update of initial state
def label_function(input: torch.Tensor) -> torch.Tensor:
    game = GameOfLife(input.int().numpy())
    game.update()
    return torch.from_numpy(game.state).reshape((-1, N_FEATURES)).float()


def input_function(natural: int) -> torch.Tensor:
    game = GameOfLife.by_natural(natural, SHAPE)
    return torch.from_numpy(game.state).float()

In [22]:
data = Data.from_natural(1_000, N_RANGE, input_function, label_function)

print(data.train_inputs[0])

tensor([[0, 1, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [1, 1, 0, 0]], dtype=torch.int8)


In [15]:
from common.visualization import LossPlot

In [16]:
import torch.nn as nn

In [17]:
class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.LazyLinear(N_FEATURES),
        )

    def forward(self, X):
        return self.net(X)


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

In [21]:
plot = LossPlot()

for epoch in range(10):  # loop over the dataset multiple times
    for input, label in zip(data.train_inputs, data.train_labels):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(input.unsqueeze(1))
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # print statistics
    plot.update(loss.item())

RuntimeError: mat1 and mat2 must have the same dtype, but got Char and Float